<a href="https://colab.research.google.com/github/fantajeon/DLPytorch1.2/blob/master/Chapter1_Loss%EC%99%80_Optimization_Package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Optimization Package로 최적화 하기#


Author: Hyeokjune Jeon (fantajeon@gmail.com)


저희가 지금까지는 다음의 Gradient Descent를 직접 구현했습니다: $$\begin{equation} 
w_{t+1} = w_{t} - \eta \frac{\partial L}{\partial w} \\
b_{t+1} = b_{t} - \eta \frac{\partial L}{\partial b}
\end{equation}$$

그런데 저희가 다른 최적화 기법을 일일이 구현할 필요 없이, 문제에 적합한 다른 최적화를 선택할 수 있습니다. Adam, AdaGrad 등이 있습니다. 부담갖지 말고 패키지처럼 사용할 수 있어서 매우 다행입니다.
패키지는 torch.optim에 다 모여있습니다.

최적화의 과정은 보통 이렇습니다.
1. optimizer 객체를 생성한다: nn.Module.paramters()인자 리스트를 생성할때 같이 넘긴다.
2. Optimizer로 각 인자들의 gradient를 0으로 초기화 한다: optim.zero_grad().
3. 값을 예측한다: $x \rightarrow y$
4. 오차를 계산한다.
5. backward()를 호출하여 모델 인자들의 grad를 계산한다.
6. 모델 인자를 갱신한다: optim.step().



In [23]:
import torch
from torch import nn

x = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6], dtype=torch.float32)
y = torch.tensor([15, 25, 40, 55, 65, 66], dtype=torch.float32)

#nn.Module화 하기
class Model(nn.Module): # 1. 상속
  def __init__(self):   # 2. 모듈 초기화 구현
    super(Model, self).__init__()
    self.w = nn.Parameter( torch.rand(1, dtype=torch.float) )   # 모델의 w 인자 정의(이런게 있군요 눈으로만 여겨 보자!)
    self.b = nn.Parameter( torch.rand(1, dtype=torch.float) )   # 모델의 b 인자 정의

  def forward(self, x): # 3. forward()구현
    return self.w * x + self.b

def loss(pred_y, y):
  return 0.5*(pred_y - y).pow(2.0).sum()

model = Model()

learning_rate = 1e-4
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

for step in range(300000):
  # PyTorch는 Dynamic하게 연산을 기록하는 기능 때문에, 
  # 매번 loop에서 x--> predict -> loss-->backward()-->gradient descent를 해줘야 함.
  pred_y = model(x)
  L = loss(pred_y, y)
  optimizer.zero_grad() # gradient[편미분을 모아 놓은 벡터] 초기화
  L.backward()
  # torch.no_grad()가 없어집니다.
  optimizer.step()

  if step % 10000 == 0:
    print("Step:{}, L:{:.5}, w={}, b={:.3}, grad(w)={:.3}, grad(b)={:.3}".format(step, L.item(), model.w.item(), model.b.item(), model.w.grad.item(), model.b.grad.item()))


Step:0, L:6945.2, w=0.24619147181510925, b=0.239, grad(w)=-1.12e+02, grad(b)=-2.64e+02
Step:10000, L:648.46, w=28.506759643554688, b=35.1, grad(w)=-13.0, grad(b)=4.32
Step:20000, L:487.27, w=40.4591178894043, b=30.8, grad(w)=-11.0, grad(b)=3.96
Step:30000, L:369.15, w=50.676658630371094, b=27.2, grad(w)=-9.44, grad(b)=3.39
Step:40000, L:282.6, w=59.422969818115234, b=24.0, grad(w)=-8.08, grad(b)=2.91
Step:50000, L:219.18, w=66.90995788574219, b=21.3, grad(w)=-6.92, grad(b)=2.49
Step:60000, L:172.7, w=73.31898498535156, b=19.0, grad(w)=-5.92, grad(b)=2.13
Step:70000, L:138.64, w=78.80545806884766, b=17.1, grad(w)=-5.07, grad(b)=1.82
Step:80000, L:113.69, w=83.50086212158203, b=15.4, grad(w)=-4.34, grad(b)=1.56
Step:90000, L:95.399, w=87.52132415771484, b=13.9, grad(w)=-3.72, grad(b)=1.34
Step:100000, L:81.998, w=90.96258544921875, b=12.7, grad(w)=-3.18, grad(b)=1.14
Step:110000, L:72.178, w=93.90826416015625, b=11.6, grad(w)=-2.72, grad(b)=0.979
Step:120000, L:64.98, w=96.43061828613281

torch.optim의 패키지를 사용하니 코드가 매우 간단해졌습니다.

# torch.optim.Optimizer #
특히 torch.optim.Optimizer로부터 상속을 받아서 구현됩니다. Optimizer에 대해서 간단히 설명해보겠습니다.
1. Optimizer 객체를 만들때, 최적화를 할 모델 인자들을 지정해줍니다. 
2. zero_grad()는 최적화될 인자들마다 grad값들을 가지는데, 이것의 값을 0으로 초기화를 합니다.
3. step()는 인자들을 갱신하라 입니다.

다른 Optimizer를 사용해 보도록 하겠습니다. 아래 코드에서 대표적으로 언급되는 Adam를 적용해 보겠습니다. 정말 간단합니다. 한 줄만으로 교체가 됩니다. 

Adam은 벌써 10000번 이내에 SGD로 했던것보다 빨리 원하는 결과를 도달했습니다. 정말 빠릅니다.

In [24]:
learning_rate = 1e-4
#optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

for step in range(300000):
  pred_y = model(x)
  L = loss(pred_y, y)
  optimizer.zero_grad()
  L.backward()
  optimizer.step()

  if step % 10000 == 0:
    print("Step:{}, L:{:.5}, w={}, b={:.3}, grad(w)={:.3}, grad(b)={:.3}".format(step, L.item(), model.w.item(), model.b.item(), model.w.grad.item(), model.b.grad.item()))


Step:0, L:45.317, w=110.48172760009766, b=5.67, grad(w)=-0.147, grad(b)=0.0548
Step:10000, L:45.239, w=111.31379699707031, b=5.37, grad(w)=-0.0201, grad(b)=1.34e-05
Step:20000, L:45.238, w=111.42850494384766, b=5.33, grad(w)=-1.19e-05, grad(b)=-1.91e-06
Step:30000, L:45.238, w=111.42857360839844, b=5.33, grad(w)=-7.15e-07, grad(b)=-3.81e-06
Step:40000, L:45.238, w=111.42857360839844, b=5.33, grad(w)=2.07e-05, grad(b)=5.34e-05
Step:50000, L:45.238, w=111.4285659790039, b=5.33, grad(w)=4.77e-07, grad(b)=3.81e-06
Step:60000, L:45.238, w=111.42857360839844, b=5.33, grad(w)=-1.55e-05, grad(b)=-4.2e-05
Step:70000, L:45.238, w=111.42857360839844, b=5.33, grad(w)=1.05e-05, grad(b)=2.86e-05
Step:80000, L:45.238, w=111.42855834960938, b=5.33, grad(w)=-2.86e-06, grad(b)=-3.81e-06
Step:90000, L:45.238, w=111.4285659790039, b=5.33, grad(w)=1.93e-05, grad(b)=4.39e-05
Step:100000, L:45.238, w=111.42857360839844, b=5.33, grad(w)=-1.55e-05, grad(b)=-4.2e-05
Step:110000, L:45.238, w=111.4285659790039, b

# Loss 패키지 사용 #
이번에는 Loss를 패키지로 활용해 보겠습니다. 지금까지 사용한 공식은 Mean Least Square Error 방식이었습니다. 이것도 PyTorch는 torch.nn.MSELoss()를 제공을 합니다. 궂이 어렵게 설계를 안하셔도 될 듯 싶습니다.


In [25]:
import torch
from torch import nn

x = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6], dtype=torch.float32)
y = torch.tensor([15, 25, 40, 55, 65, 66], dtype=torch.float32)

class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.w = nn.Parameter( torch.rand(1, dtype=torch.float) )
    self.b = nn.Parameter( torch.rand(1, dtype=torch.float) )

  def forward(self, x): # 3. forward()구현
    return self.w * x + self.b

loss = nn.MSELoss(reduction='sum')
model = Model()

learning_rate = 1e-4
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

for step in range(300000):
  pred_y = model(x)
  L = loss(pred_y, y)
  optimizer.zero_grad()
  L.backward()
  optimizer.step()

  if step % 10000 == 0:
    print("Step:{}, L:{:.5}, w={}, b={:.3}, grad(w)={:.3}, grad(b)={:.3}".format(step, L.item(), model.w.item(), model.b.item(), model.w.grad.item(), model.b.grad.item()))

Step:0, L:1.3662e+04, w=0.9764900207519531, b=0.394, grad(w)=-2.22e+02, grad(b)=-5.24e+02
Step:10000, L:963.68, w=40.89741516113281, b=30.7, grad(w)=-21.9, grad(b)=7.88
Step:20000, L:559.36, w=59.74451446533203, b=23.9, grad(w)=-16.1, grad(b)=5.77
Step:30000, L:342.26, w=73.55522918701172, b=18.9, grad(w)=-11.8, grad(b)=4.23
Step:40000, L:225.68, w=83.6752700805664, b=15.3, grad(w)=-8.63, grad(b)=3.1
Step:50000, L:163.08, w=91.09107208251953, b=12.6, grad(w)=-6.32, grad(b)=2.27
Step:60000, L:129.46, w=96.52522277832031, b=10.7, grad(w)=-4.63, grad(b)=1.66
Step:70000, L:111.41, w=100.50750732421875, b=9.26, grad(w)=-3.39, grad(b)=1.22
Step:80000, L:101.72, w=103.42494201660156, b=8.21, grad(w)=-2.49, grad(b)=0.896
Step:90000, L:96.517, w=105.56201934814453, b=7.44, grad(w)=-1.82, grad(b)=0.654
Step:100000, L:93.72, w=107.12969207763672, b=6.88, grad(w)=-1.34, grad(b)=0.484
Step:110000, L:92.222, w=108.27526092529297, b=6.47, grad(w)=-0.981, grad(b)=0.351
Step:120000, L:91.415, w=109.116

## MSELoss말고 다른 것은? ##
정말 많이 있습니다. [Loss를 메뉴얼](https://pytorch.org/docs/stable/nn.html#loss-functions) 여기를 클릭하여 방문 하시면, 정말 많이 있습니다. 본 코드에서는 torch.nn.L1Loss를 사용해 보겠습니다.

L1Loss는 차이의 절대값의 합을 손실로 정의한 함수입니다: $$ loss(\hat {y}, y) = \sum { |\hat{y} - y| }$$.

In [32]:
loss = nn.L1Loss(reduction='sum')
model = Model()

learning_rate = 1e-4
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

for step in range(300000):
  pred_y = model(x)
  L = loss(pred_y, y)
  optimizer.zero_grad()
  L.backward()
  optimizer.step()

  if step % 10000 == 0:
    print("Step:{}, L:{:.5}, w={}, b={:.3}, grad(w)={:.3}, grad(b)={:.3}".format(step, L.item(), model.w.item(), model.b.item(), model.w.grad.item(), model.b.grad.item()))

Step:0, L:259.0, w=0.9472194314002991, b=0.837, grad(w)=-2.1, grad(b)=-6.0
Step:10000, L:218.59, w=3.0476720333099365, b=6.84, grad(w)=-2.1, grad(b)=-6.0
Step:20000, L:178.19, w=5.146836280822754, b=12.8, grad(w)=-2.1, grad(b)=-6.0
Step:30000, L:153.05, w=7.097900390625, b=17.4, grad(w)=-1.9, grad(b)=-4.0
Step:40000, L:133.42, w=8.995712280273438, b=21.4, grad(w)=-1.9, grad(b)=-4.0
Step:50000, L:121.59, w=10.66004753112793, b=24.2, grad(w)=-1.5, grad(b)=-2.0
Step:60000, L:115.34, w=12.157316207885742, b=26.2, grad(w)=-1.5, grad(b)=-2.0
Step:70000, L:109.09, w=13.654584884643555, b=28.2, grad(w)=-1.5, grad(b)=-2.0
Step:80000, L:102.84, w=15.151853561401367, b=30.2, grad(w)=-1.5, grad(b)=-2.0
Step:90000, L:96.583, w=16.653257369995117, b=32.2, grad(w)=-1.5, grad(b)=-2.0
Step:100000, L:90.356, w=18.160062789916992, b=34.2, grad(w)=-1.5, grad(b)=-2.0
Step:110000, L:88.77, w=19.144163131713867, b=34.5, grad(w)=-0.9, grad(b)=0.0
Step:120000, L:87.964, w=20.040616989135742, b=34.5, grad(w)=-0

이것은 MSELoss()보다 학습 능력이 떨어집니다. Loss가 설계되지 못한 것으로 판단됩니다. Loss의 문제일까요? 아니면 저 L1Loss에 맞는 지금의 learning rate=1e-4가 잘 못 된 것일까요? 적합한 Loss를 찾는 것도 굉장히 중요하고, 적당한 learning rate를 설정하는 것도 중요합니다. 이건 많은 경험을 통해서 체득해야 합니다.

상기 코드를 learning_rate를 바꿔보면서 실험해보면서 관찰을 해보시길 바랍니다.

In [30]:
loss = nn.SmoothL1Loss(reduction='sum')
model = Model()

learning_rate = 1e-3
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

for step in range(300000):
  pred_y = model(x)
  L = loss(pred_y, y)
  optimizer.zero_grad()
  L.backward()
  optimizer.step()

  if step % 10000 == 0:
    print("Step:{}, L:{:.5}, w={}, b={:.3}, grad(w)={:.3}, grad(b)={:.3}".format(step, L.item(), model.w.item(), model.b.item(), model.w.grad.item(), model.b.grad.item()))

Step:0, L:257.76, w=0.7583906650543213, b=0.61, grad(w)=-2.1, grad(b)=-6.0
Step:10000, L:176.75, w=10.759114265441895, b=10.6, grad(w)=-2.1, grad(b)=-6.0
Step:20000, L:113.1, w=20.655839920043945, b=20.2, grad(w)=-1.84, grad(b)=-3.69
Step:30000, L:78.533, w=30.416183471679688, b=29.4, grad(w)=-1.5, grad(b)=-2.0
Step:40000, L:67.113, w=39.874900817871094, b=31.1, grad(w)=-0.9, grad(b)=0.0
Step:50000, L:58.118, w=49.869407653808594, b=31.1, grad(w)=-0.9, grad(b)=0.0
Step:60000, L:49.123, w=59.863914489746094, b=29.1, grad(w)=-0.9, grad(b)=1.53e-05
Step:70000, L:40.128, w=69.8584213256836, b=23.1, grad(w)=-0.9, grad(b)=6.1e-05
Step:80000, L:31.133, w=79.8529281616211, b=17.1, grad(w)=-0.9, grad(b)=0.0
Step:90000, L:24.576, w=89.30147552490234, b=12.8, grad(w)=-0.481, grad(b)=5.34e-05
Step:100000, L:21.384, w=98.7210693359375, b=9.38, grad(w)=-0.3, grad(b)=0.0
Step:110000, L:18.386, w=108.715576171875, b=6.38, grad(w)=-0.3, grad(b)=0.0
Step:120000, L:15.568, w=118.51985168457031, b=3.8, gr